In [9]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from utils import show_rank_metrics


In [10]:
# train = pd.read_csv('train.csv', index_col=0).sample(frac=1)
# y_train = train['correct']
# X_train = train.drop(columns=['name', 'firstChar', 'group', 'correct'])
#
# test = pd.read_csv('test.csv', index_col=0)
# y_test = test['correct']
# X_test = test.drop(columns=['name', 'firstChar', 'group', 'correct'])
train = pd.read_csv('train_cat.csv', index_col=0).sample(frac=1)
test = pd.read_csv('test_cat.csv', index_col=0)
train.fillna('', inplace=True)
test.fillna('', inplace=True)


y_train = train['correct']
X_train = train.drop(columns=['firstChar', 'group', 'correct'])

y_test = test['correct']
X_test = test.drop(columns=['firstChar', 'group', 'correct'])

# df_valid_cat.fillna('', inplace=True)

/tmp/ipykernel_21005/2614602940.py:9: DtypeWarning: Columns (20,35) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_cat.csv', index_col=0)


In [11]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(verbose=False, random_seed=0, task_type="CPU")
model.fit(X_train, y_train, cat_features=X_train.columns.tolist())

KeyboardInterrupt: 

In [ ]:
#model.predict_proba(X_test)

In [ ]:
show_rank_metrics(
    df_group=pd.DataFrame(data=train['group'].tolist(), columns=['group']),
    df_proba=pd.DataFrame(data=model.predict_proba(X_train)[:,1], columns=['proba']),
    df_true=pd.DataFrame(data=y_train.tolist(), columns=['correct'])
)

In [ ]:
show_rank_metrics(
    df_group=pd.DataFrame(data=test['group'].tolist(), columns=['group']),
    df_proba=pd.DataFrame(data=model.predict_proba(X_test)[:,1], columns=['proba']),
    df_true=pd.DataFrame(data=y_test.tolist(), columns=['correct'])
)

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
model.save_model("kek")